# Name of the project

In [11]:
# Dependencies and Setup
import pandas as pd

# import matplotlib.pyplot as plt
# import scipy.stats as st
# from scipy.stats import linregress

# Study data files
country_data_path = "Data/final.csv"
doctors_data_path = "Data/Medical_doctors.csv"
population_data_path = "Data/"

# Read the mouse data and the study results
country_data = pd.read_csv(country_data_path)
doctors_data = pd.read_csv(doctors_data_path)
population_data = pd.read_csv(population_data_path)



In [12]:
# Display "final" file
country_data

,Country,Year,Gender,Life expectancy,Unemployment,Infant Mortality,GDP,GNI,Clean fuels and cooking technologies,Per Capita,...,DPT Immunization,HepB3 Immunization,Measles Immunization,Hospital beds,Basic sanitation services,Tuberculosis treatment,Urban population,Rural population,Non-communicable Mortality,Sucide Rate
0,Afghanistan,2000,Female,57.120,13.846,85.0,1.368449e+10,1.824044e+10,6.400000,434.739537,...,24.0,65.076923,27.0,0.30,21.870802,85.0,22.078,77.922,39.4,4.8
1,Afghanistan,2001,Female,57.596,13.849,82.7,1.368449e+10,1.824044e+10,7.200000,434.739537,...,33.0,65.076923,37.0,0.39,21.881304,84.0,22.169,77.831,40.1,4.8
2,Afghanistan,2002,Female,58.080,14.355,80.4,4.055180e+09,1.824044e+10,8.200000,179.426579,...,36.0,65.076923,35.0,0.39,23.274193,87.0,22.261,77.739,40.5,4.6
3,Afghanistan,2003,Female,58.578,14.222,77.8,4.515559e+09,1.824044e+10,9.400000,190.683814,...,41.0,65.076923,39.0,0.39,24.667435,86.0,22.353,77.647,40.1,4.6
4,Afghanistan,2004,Female,59.093,14.047,75.3,5.226779e+09,1.824044e+10,10.400000,211.382074,...,50.0,65.076923,48.0,0.39,26.067695,89.0,22.500,77.500,40.2,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,Zimbabwe,2015,Male,57.778,4.352,46.6,1.996312e+10,1.959700e+10,29.799999,1445.069702,...,87.0,87.000000,86.0,2.35,38.120646,81.0,32.385,67.615,32.5,23.8
9924,Zimbabwe,2016,Male,58.565,4.358,45.1,2.054868e+10,2.015340e+10,29.700001,1464.588957,...,90.0,90.000000,95.0,2.35,37.529193,81.0,32.296,67.704,31.8,22.5
9925,Zimbabwe,2017,Male,59.105,4.353,44.2,1.758489e+10,1.727577e+10,29.600000,1235.189032,...,89.0,89.000000,90.0,2.35,36.941673,83.0,32.237,67.763,30.9,21.0
9926,Zimbabwe,2018,Male,59.501,4.360,42.9,1.811554e+10,1.781019e+10,29.799999,1254.642265,...,89.0,89.000000,88.0,2.35,36.357160,84.0,32.209,67.791,30.3,19.9


In [13]:
# Display "Medical_doctors" file
doctors_data

,Country,Year,Medical doctors (per 10 000 population),Medical doctors (number),Generalist medical practitioners (number),Specialist medical practitioners (number),Medical doctors not further defined (number)
0,Afghanistan,2020,2.538,9880.0,NaN,NaN,9880.0
1,Afghanistan,2019,2.125,8082.0,NaN,NaN,8082.0
2,Afghanistan,2018,2.966,11026.0,NaN,NaN,11026.0
3,Afghanistan,2017,2.409,8744.0,NaN,NaN,8744.0
4,Afghanistan,2016,2.782,9842.0,NaN,NaN,9842.0
...,...,...,...,...,...,...,...
3578,Zimbabwe,2004,1.735,2086.0,NaN,NaN,2086.0
3579,Zimbabwe,2000,1.262,1499.0,1066.0,297.0,136.0
3580,Zimbabwe,1995,1.428,1630.0,NaN,NaN,1630.0
3581,Zimbabwe,1990,1.265,1320.0,NaN,NaN,1320.0


In [14]:
country_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9928 entries, 0 to 9927
Data columns (total 22 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Country                                  9928 non-null   object 
 1   Year                                     9928 non-null   int64  
 2   Gender                                   9928 non-null   object 
 3   Life expectancy                          9928 non-null   float64
 4   Unemployment                             9928 non-null   float64
 5   Infant Mortality                         9928 non-null   float64
 6   GDP                                      9928 non-null   float64
 7   GNI                                      9928 non-null   float64
 8   Clean fuels and cooking technologies     9928 non-null   float64
 9   Per Capita                               9928 non-null   float64
 10  Mortality caused by road traffic injury  9928 no

In [16]:
doctors_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3583 entries, 0 to 3582
Data columns (total 7 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Country                                       3583 non-null   object 
 1   Year                                          3583 non-null   int64  
 2   Medical doctors (per 10 000 population)       2844 non-null   float64
 3   Medical doctors (number)                      3410 non-null   float64
 4   Generalist medical practitioners (number)     1279 non-null   float64
 5   Specialist medical practitioners (number)     1195 non-null   float64
 6   Medical doctors not further defined (number)  2780 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 196.1+ KB


In [41]:
print(f"number of countries in the final file is {country_data['Country'].nunique()}")
print(f"number of countries in the Medical_doctors file is {doctors_data['Country'].nunique()}")
print(f"number of years in the final file is {country_data['Year'].nunique()}, the earliest is {country_data['Year'].min()}, the latest is {country_data['Year'].max()}")
print(f"number of years in the Medical_doctors file is {doctors_data['Year'].nunique()}, the earliest is {doctors_data['Year'].min()}, the latest is {doctors_data['Year'].max()}")

number of countries in the final file is 256
number of countries in the Medical_doctors file is 196
number of years in the final file is 20, the earliest is 2000, the latest is 2019
number of years in the Medical_doctors file is 69, the earliest is 1952, the latest is 2020


In [42]:
# Calculate total for final (Male+Female)

# Filter on countries that aren't present in one of the lists ()

# remove unnecessary columns

# list the countries that were filtered out (list the countires from merged list, loop through both original lists and add not present to the list to print)

# compare that all the countries have all the year's info to have enough information for comparison. 

In [59]:
# Define the list of the years to be included into the analysis
first_year = 2015
last_year = 2019
years_list = list(range(first_year,last_year + 1, 1))

In [60]:
doctors_data_recent = doctors_data[doctors_data["Year"].isin(years_list)]
doctors_data_recent

,Country,Year,Medical doctors (per 10 000 population),Medical doctors (number),Generalist medical practitioners (number),Specialist medical practitioners (number),Medical doctors not further defined (number)
1,Afghanistan,2019,2.125,8082.0,NaN,NaN,8082.0
2,Afghanistan,2018,2.966,11026.0,NaN,NaN,11026.0
3,Afghanistan,2017,2.409,8744.0,NaN,NaN,8744.0
4,Afghanistan,2016,2.782,9842.0,NaN,NaN,9842.0
5,Afghanistan,2015,2.850,9808.0,NaN,NaN,9808.0
...,...,...,...,...,...,...,...
3557,Zambia,2016,0.925,1514.0,NaN,NaN,1514.0
3565,Zimbabwe,2018,2.096,3026.0,1863.0,619.0,544.0
3566,Zimbabwe,2017,1.859,2646.0,1754.0,446.0,446.0
3567,Zimbabwe,2016,1.788,2508.0,1754.0,446.0,308.0


In [61]:
country_data_recent = country_data[country_data["Year"].isin(years_list)]
country_data_recent

,Country,Year,Gender,Life expectancy,Unemployment,Infant Mortality,GDP,GNI,Clean fuels and cooking technologies,Per Capita,...,DPT Immunization,HepB3 Immunization,Measles Immunization,Hospital beds,Basic sanitation services,Tuberculosis treatment,Urban population,Rural population,Non-communicable Mortality,Sucide Rate
15,Afghanistan,2015,Female,64.877,14.094,49.6,1.913421e+10,1.930719e+10,28.600000,556.007221,...,65.0,65.0,63.0,0.500000,42.626023,88.0,24.803,75.197,35.7,3.6
16,Afghanistan,2016,Female,65.275,14.084,47.7,1.811656e+10,1.829710e+10,30.299999,512.012778,...,66.0,66.0,64.0,0.500000,44.201217,93.0,25.020,74.980,36.0,3.6
17,Afghanistan,2017,Female,65.656,14.063,46.1,1.875347e+10,1.900025e+10,32.200001,516.679862,...,66.0,66.0,64.0,0.390000,45.789127,91.0,25.250,74.750,35.9,3.4
18,Afghanistan,2018,Female,66.026,14.012,44.5,1.805323e+10,1.824084e+10,34.099998,485.668419,...,66.0,66.0,64.0,0.432222,47.390662,91.0,25.495,74.505,36.0,3.5
19,Afghanistan,2019,Female,66.388,14.065,42.9,1.879945e+10,1.909831e+10,36.000000,494.179350,...,66.0,66.0,64.0,0.432222,49.006170,91.0,25.754,74.246,36.2,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9923,Zimbabwe,2015,Male,57.778,4.352,46.6,1.996312e+10,1.959700e+10,29.799999,1445.069702,...,87.0,87.0,86.0,2.350000,38.120646,81.0,32.385,67.615,32.5,23.8
9924,Zimbabwe,2016,Male,58.565,4.358,45.1,2.054868e+10,2.015340e+10,29.700001,1464.588957,...,90.0,90.0,95.0,2.350000,37.529193,81.0,32.296,67.704,31.8,22.5
9925,Zimbabwe,2017,Male,59.105,4.353,44.2,1.758489e+10,1.727577e+10,29.600000,1235.189032,...,89.0,89.0,90.0,2.350000,36.941673,83.0,32.237,67.763,30.9,21.0
9926,Zimbabwe,2018,Male,59.501,4.360,42.9,1.811554e+10,1.781019e+10,29.799999,1254.642265,...,89.0,89.0,88.0,2.350000,36.357160,84.0,32.209,67.791,30.3,19.9


In [62]:
# Combine the data into a single DataFrame
country_doctors = pd.merge(country_data_recent,doctors_data_recent, on= ["Country", "Year"], how="inner")

# Display the data table for preview
country_doctors

,Country,Year,Gender,Life expectancy,Unemployment,Infant Mortality,GDP,GNI,Clean fuels and cooking technologies,Per Capita,...,Tuberculosis treatment,Urban population,Rural population,Non-communicable Mortality,Sucide Rate,Medical doctors (per 10 000 population),Medical doctors (number),Generalist medical practitioners (number),Specialist medical practitioners (number),Medical doctors not further defined (number)
0,Afghanistan,2015,Female,64.877,14.094,49.6,1.913421e+10,1.930719e+10,28.600000,556.007221,...,88.0,24.803,75.197,35.7,3.6,2.850,9808.0,NaN,NaN,9808.0
1,Afghanistan,2015,Male,61.965,10.420,56.5,1.913421e+10,1.930719e+10,28.600000,556.007221,...,88.0,24.803,75.197,35.4,4.3,2.850,9808.0,NaN,NaN,9808.0
2,Afghanistan,2016,Female,65.275,14.084,47.7,1.811656e+10,1.829710e+10,30.299999,512.012778,...,93.0,25.020,74.980,36.0,3.6,2.782,9842.0,NaN,NaN,9842.0
3,Afghanistan,2016,Male,62.343,10.412,54.6,1.811656e+10,1.829710e+10,30.299999,512.012778,...,93.0,25.020,74.980,35.1,4.4,2.782,9842.0,NaN,NaN,9842.0
4,Afghanistan,2017,Female,65.656,14.063,46.1,1.875347e+10,1.900025e+10,32.200001,516.679862,...,91.0,25.250,74.750,35.9,3.4,2.409,8744.0,NaN,NaN,8744.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,Zimbabwe,2016,Male,58.565,4.358,45.1,2.054868e+10,2.015340e+10,29.700001,1464.588957,...,81.0,32.296,67.704,31.8,22.5,1.788,2508.0,1754.0,446.0,308.0
982,Zimbabwe,2017,Female,62.220,5.192,35.4,1.758489e+10,1.727577e+10,29.600000,1235.189032,...,83.0,32.237,67.763,28.3,9.6,1.859,2646.0,1754.0,446.0,446.0
983,Zimbabwe,2017,Male,59.105,4.353,44.2,1.758489e+10,1.727577e+10,29.600000,1235.189032,...,83.0,32.237,67.763,30.9,21.0,1.859,2646.0,1754.0,446.0,446.0
984,Zimbabwe,2018,Female,62.598,5.206,34.4,1.811554e+10,1.781019e+10,29.799999,1254.642265,...,84.0,32.209,67.791,27.4,8.7,2.096,3026.0,1863.0,619.0,544.0


In [63]:
country_doctors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986 entries, 0 to 985
Data columns (total 27 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Country                                       986 non-null    object 
 1   Year                                          986 non-null    int64  
 2   Gender                                        986 non-null    object 
 3   Life expectancy                               986 non-null    float64
 4   Unemployment                                  986 non-null    float64
 5   Infant Mortality                              986 non-null    float64
 6   GDP                                           986 non-null    float64
 7   GNI                                           986 non-null    float64
 8   Clean fuels and cooking technologies          986 non-null    float64
 9   Per Capita                                    986 non-null    flo

In [64]:
print(f"number of countries in the merged data frame is {country_doctors['Country'].nunique()}")
print(f"number of years in the merged data frame is {country_doctors['Year'].nunique()}")
print(f"number of rows in the merged data frame is {country_doctors.shape[0]}, and supposed to be {country_doctors['Country'].nunique()*country_doctors['Year'].nunique()*2}")

number of countries in the merged data frame is 150
number of years in the merged data frame is 5
number of rows in the merged data frame is 986, and supposed to be 1500
